<a href="https://colab.research.google.com/github/Samuel-CHLam/Oxford_Simulation_Methods_and_Stochastic_Algorithms/blob/main/Question_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 2 (by Olivia Pricilia)

Now consider a European call option in which the final value of the underlying is 

$S(T)=S(0) exp((r−0.5σ^2)T+σW(T))$

where

$W(T)= \sqrt{T}X= \sqrt{T} \Phi^{-1}(U)$

with X being a unit Normal, or U a uniform (0, 1) random variable.

The payoff function is:

$f(S) = exp(−rT) (S(T) − K)^{+}$

and the constants are r = 0.05, σ = 0.2, S(0) = 100, K = 100.

The analytic value is given by the routine european call available from the course webpage; read its header to see how to call it.
(There is no need to compute the analytic variance as in part a) in the previous question; just use the unbiased estimator.)

Investigate the following forms of variance reduction:





### Import European_Call Function from webpage

In [1]:
import numpy as np

from numpy       import pi, sqrt, log, exp
from scipy.stats import norm

#
# Normal cumulative distribution function, with extension
# for complex argument with small imaginary component
#

def norm_cdf(x):
    if not isinstance(x, np.ndarray):
        xr = x.real
        xi = x.imag
        if abs(xi) > 1.0e-10:
            raise ValueError('imag(x) too large in norm_cdf(x)')

        ncf = norm.cdf(xr)
        if abs(xi) > 0:
            ncf = ncf + 1.0j*xi*norm.pdf(xr)
    else:
        xr = np.real(x)
        xi = np.imag(x)
        if any(abs(xi) > 1.0e-10):
            raise ValueError('imag(x) too large in norm_cdf(x)')

        ncf = norm.cdf(xr)
        if any(abs(xi) > 0):
            ncf = ncf + 1.0j*xi*norm.pdf(xr)

    return ncf

# V = european_call(r,sigma,T,S,K,opt)
#
# Black-Scholes European call option solution
# as defined in equation (3.17) on page 48 of 
# The Mathematics of Financial Derivatives
# by Wilmott, Howison and Dewynne
#
# r     - interest rate
# sigma - volatility
# T     - time interval
# S     - asset value(s)  (float or flattened numpy array)
# K     - strike price(s) (float or flattened numpy array)
# opt   - 'value', 'delta', 'gamma' or 'vega'
# V     - option value(s) (float or flattened numpy array)
#

def european_call(r,sigma,T,S,K,opt):

    S  = S + 1.0e-100     # avoids problems with S=0
    K  = K + 1.0e-100     # avoids problems with K=0

    d1 = ( log(S) - log(K) + (r+0.5*sigma**2)*T ) / (sigma*sqrt(T))
    d2 = ( log(S) - log(K) + (r-0.5*sigma**2)*T ) / (sigma*sqrt(T))

    if opt == 'value':
        V = S*norm_cdf(d1) - exp(-r*T)*K*norm_cdf(d2)
    elif opt == 'delta':
        V = norm_cdf(d1)
    elif opt == 'gamma':
        V = exp(-0.5*d1**2) / (sigma*sqrt(2*pi*T)*S)
    elif opt == 'vega':
        V =             S*(exp(-0.5*d1**2)/sqrt(2*pi))*( sqrt(T)-d1/sigma) \
            - exp(-r*T)*K*(exp(-0.5*d2**2)/sqrt(2*pi))*(-sqrt(T)-d2/sigma)

    else:
        raise ValueError('invalid value for opt -- must be "value", "delta", "gamma", "vega"')

    return V


In [2]:
np.exp(1)

2.718281828459045

##2 a. 
First, try antithetic variables using $\frac{1}{2} (f (W ) + f (−W ))$ where W is the value of the underlying Brownian motion at maturity.
What is the estimated correlation between $f(W)$ and $f(−W)$? How much variance reduction does this give?


**Answer:**

Correlation of f(W) and f(-W) is -0.5011364822986971 (i.e. around 0.50). 
And using the anthtetic variables results in the variance reduction to around 25% of the original variance
(i.e. 0.24943175885065147). (See the code cell below for details)


In [3]:
from IPython.core.display import set_matplotlib_close
import numpy as np
import math 

#General input:

r = 0.05
sigma = 0.2
S0 = 100.0
K = 100.0
T=1.0
N = 10**6 #as in Q1

#U = np.random.uniform(0.0,1.0,N)  # U ~ Unif(0,1), X~N(0,1)

#european_call(r=r,sigma=sigma,T=T,S=S0,K=K,opt='value') 
#price of european call option
  
X =np.random.randn(N,1)
WT = sqrt(T)*X #np.random.randn(N,1) #W(T) = sqrt(T)*X = sqrt(T)*inverse_cdf(U)

#final price and payoff
def S_T(S0, r, sigma, T, WT):
  return S0 * np.exp((r-0.5*(sigma**2))*T + sigma * WT)

def f_ST(r, T, ST, K):
  return np.exp(-r*T)*np.maximum(0.0, ST - K)

ST_pos= S_T(S0, r, sigma, T, WT)
ST_neg = S_T(S0, r, sigma, T, -WT)

fS_pos = f_ST(r, T, ST_pos, K) #f(W)
fS_neg = f_ST(r, T, ST_neg, K) #f(-W)


##########antithetic variables
fS_mean = np.sum(fS_pos + fS_neg)/(2.0*N)


#correlation of f(W) and f(-W):
#var(f(W))= var(f(-W)) 
var0 = np.sum(fS_pos**2 + fS_neg**2)/(2.0*N) - fS_mean**2
cov0 = np.sum((fS_pos -fS_mean)*(fS_neg - fS_mean))/(N)
cor0 = cov0/var0
print('correlation of f(W) and f(-W) is {}'.format(cor0))

### variance reduction
var_new = 0.5*(var0 + cov0)
var_pct = var_new / var0
print('variance reduction is {}'.format(var_pct))



correlation of f(W) and f(-W) is -0.501536094707017
variance reduction is 0.24923195264649153



## 2 b.

Second, try using $exp(−rT)S(T)$ as a control variate, noting that its expected value is $S(0)$.
Again, how much variance reduction does this give?


**Answer**:

Using control variates specified, we calculate the correlation of f (original payoff) and g (control variates), by first calculating their covariances and variances. Then, we calculate the new variance, which is:
$\frac{VAR(f)}{N} * (1- corr(f,g)^2)$

the resulting variance becomes 14.5% (i.e. 0.14502356901851332) of the original variance (see below for the code).


In [4]:
#control variates: (note we re-use some variables in 2a)
gS = np.exp(-r*T)*ST_pos 

fS_mcmean = np.sum(fS_pos)/N
gS_mcmean = np.sum(gS)/N
gS_exp = S0

#calculate optimum value of lambda
cov_fg = np.sum((fS_pos-fS_mcmean)*(gS-gS_mcmean))/N
var_f = np.sum((fS_pos-fS_mcmean)**2)/N 
var_g = np.sum((gS-gS_mcmean)**2)/N 
corr_fg = cov_fg/(np.sqrt(var_f*var_g))

var_fg_new = var_f  * (1.- corr_fg**2)
var_fg_pct = var_fg_new / var_f


print('variance reduction is {}'.format(var_fg_pct))


#define new estimator
k = cov_fg / var_g
fS_hat = fS_mcmean - k*(gS_mcmean - S0)


variance reduction is 0.14527567048979928


## 2. c. 

For the case of a digital put option,
$P =exp(−rT)H(K−S(T))$

where $H(x)$ is the Heaviside step function, with parameters r = 0.05, σ = 0.2, T = 1, S(0) = 100 ,K = 50, investigate the use of importance sampling:

### 2 c. i.

First, estimate the value without importance sampling.
How many samples are needed to obtain a value which is correct to within 10%? (i.e. the 3 standard deviation confidence limit corresponds to ± 10%).


**Answer:**

First we calculate the mean $P_{mcmean}$ and variance $var_P$of digital Put.

We want 3 std.dev confidence limit corresponds to +/- 10%, thus we want:
$3* \sigma_{P} / \sqrt{N_{solve}} = 0.1*P_{mcmean} $


Hence, solving for N_solve, we need 6205997 samples (rounding up from 6205996.551724128)

In [5]:

#payoff for digital put:
r     = 0.05
sigma = 0.2
T     = 1.0
K     = 50.0
S0    = 100.0

ST = S_T(S0, r, sigma, T, WT)


P = exp(-r*T)*(1.+ np.sign(K-ST))/2.0 #rewrite Heavyside function 
P_mcmean = np.sum(P) /N
var_P = np.sum(P**2)/N - (P_mcmean)**2 

#we want 3 std.dev confidence limit corresponds to +/- 10%
# thus we want  3* std.dev(P) / sqrt(N_solve) = 0.1*P_mcmean 
N_solve = (3.*np.sqrt(var_P)/ (0.1*P_mcmean) )**2


print('Number of samples needed : {}'.format(N_solve))

Number of samples needed : 5805551.612903213


### 2.c.ii.

Second, try using importance sampling, adjusting the drift
(i.e. changing the $(r−0.5σ^{2})T)$ term to a different constant) so that half of the samples are below the strike K, and the other half are above. Now how many samples are required to get the value correct to within 10%?


**Answer:***

With drift $\mu_{old} = (r−0.5σ^{2})T$, we have $S(T)=S(0) exp((r−0.5σ^2)T+σW(T))$ and so, $\log (\frac{S(T)}{S(0)})$ ~ $N( \mu_{old} , \sigma^{2} {T})$. 

Now, we want half of the samples of S(T) are below the strike K and the other half are above, hence this translates to median of S(T) is K, and so (since mean = median for normally distributed random variable), we can deduce that $\log (\frac{S(T)}{S(0)})$ has adjusted drift (mean) of $\mu_{new} = \log(\frac{K}{S(0)})$ .


Now, the Radon Nikodym Derivatives is:
$R(x) =  \frac{p_{old}(x)}{p_{new}(x)}$ , where $p_{old}(x), p_{new}(x)$ are the p.d.f. of lognormal distribution with drift $\mu_{old}, \mu_{new}$ respectively, and is given by:

$p_{old}(x) = \frac{1}{x (\sigma \sqrt{T}) \sqrt{2 \pi}} exp(- \frac{(ln(x) - \mu_{old} )^2}{2 \sigma^{2} T}) $ and $p_{new}(x) = \frac{1}{x (\sigma \sqrt{T}) \sqrt{2 \pi}} exp(- \frac{(ln(x) - \mu_{new} )^2}{2 \sigma^{2} T})$ 

Hence, (taking $x = S(T)/S(0) = exp(\mu_{old} + \sigma W(T))$)

$R(x) = exp(- \frac{ 2 \ln(x) ( \mu_{new} - \mu_{old} ) + ( \mu^{2}_{old} - \mu^{2}_{new} )}{2 \sigma^{2} T})= exp(- \frac{ (2 (\mu_{new} + \sigma W(T)) - \mu_{new} -\mu_{old} )  ( \mu_{new} - \mu_{old} ) }{2 \sigma^{2} T}) = exp(- \frac{ ( \mu_{new} -\mu_{old} +2 \sigma W(T)) )  ( \mu_{new} - \mu_{old} ) }{2 \sigma^{2} T})$

Now, calculate the value of Monte-Carlo Mean ($PR_{mcmean}$) of Payoff P with new drift of S and multiply by Radon-Nikodym Derivatives R, and also its variance $var_{PR}$.


As before, we want 3 std.dev confidence limit corresponds to +/- 10%, thus we want:
$3* \sigma_{PR} / \sqrt{N_{newsolve}} = 0.1*PR_{mcmean} $


Hence, solving for $N_{newsolve}$, we need 3672 samples (rounding up from 3671.1034603785765), which is only 0.06% of the number of samples needed without using importance of sampling (as in 2c (i)).

In [6]:
# Note that since S follow log-normal distribution, we need to change the drift to log(K/S0)= log(1/2) so that it's 

drift_new = log(K /S0)
drift_old = (r- 0.5* sigma*sigma)*T

#R(x) = p_old(x)/ p_new(x) = exp() radon nikodym derivatives

R = exp(-(drift_new-drift_old + 2.*sigma*WT)*(drift_new - drift_old)/ (2.0 * sigma * sigma *T))

#S(T) with new drift log(K/S0)
ST_new = K * exp(sigma * WT)

#Payoff P with new drift of S and multiply by Radon-Nikodym Derivatives R:
PR_new = exp(-r*T)*(1.+ np.sign(K-ST_new))/2.0 * R

PR_new_mcmean = np.sum(PR_new) /N
var_PR_new = np.sum(PR_new**2)/N - (PR_new_mcmean)**2 

#we want 3 std.dev confidence limit corresponds to +/- 10%
# thus we want  3* std.dev(P_new) / sqrt(N_new_solve) = 0.1*P_new_mcmean 
N_new_solve = (3.*np.sqrt(var_PR_new)/ (0.1*PR_new_mcmean) )**2


print('Number of samples needed : {}'.format(N_new_solve))

Number of samples needed : 3659.0553996907356
